# Homografía y videos de secciones delgadas

In [ ]:

import cv2 # open-cv versión en python de libreria de visión artificial de c++
import numpy as np # libreria para tratamiento de matrices (en este caso imagenes)
import os # libreria para manejo y creacion de archivos del sistema operativo
import imageio, imutils # tratamiento de imagenes, para hacer los gifs 

# NOMBRE DEL VIDEO (sin extensión) EN ESTE CASO: flogopita 
name = "flogopita" # creará un directorio llamado de esta forma para guardar fotogramas y resultados de la homografía

El proceso de Homografía toma dos imagenes y genera puntos de relación entre ellas, luego las alinea haciendo que estos cuadren, para tener dos imagenes alineadas de un mismo objeto 

![alt text](https://i.stack.imgur.com/TPg4I.png "Ejemplo de que es el proceso de Homografía, tomado de DeTone, 2016")

Las lineas conectan los puntos en común entre las dos imagenes

In [ ]:
from IPython.display import Video

Video(name+".mp4",width=600, height=400,html_attributes = "controls muted")

## Definición de funciones

### Desglosar en Fotogramas


In [ ]:


def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    
    if hasFrames:

        cv2.imwrite(name+"/image"+str(count)+".jpg", image)     # guadar fotogramas como jpg
    return hasFrames

vidcap = cv2.VideoCapture(name+'.mp4')

try:
    os.mkdir(name)
except OSError:
    print ("Fallo creacion de directorio: %s --- ¿ya existe?" % name)
else:
    print ("Se creo directorio:  %s " % name)
   
sec = 0
frameRate = 0.2 # AQUI se modifica la tasa  de los fotogramas, más bajo = más fotogramas
count=1
success = getFrame(sec)
while success: # para de extraer fotogramas cuando ya no hay video
    count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)

### Hacer Homografía

Se define una función para simplificar la extracción de la homografía entre 2 imagenes

In [ ]:
def hacerHomografia(img2,im1,num: int,name, nOrbs=200):
    
    """
    INPUT
    img2 -> imagen de referencia
    im1 ->  imagen que va ser alineada
    name ->  nombre de la carpeta donde estan los fotogramas: string
    num -> numero en el loop, para guardar acordemente los resultados
    orbs ->  numero de Orbs (puntos de referencia seleccionados automáticamente)
    
    OUTPUT
    guarda la imagen 1 alineada respecto a objetos de la imagen 2
    
    """

    img1 = cv2.cvtColor(im1,cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(nOrbs)
        #  encontrar los objetos de referencia ORBS
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)

    # emparejar los ORBS y organizar
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    matches =  matcher.match(des1, des2, None)
    matches = sorted(matches, key = lambda x: x.distance) #Organizar los emparejamientos

    # Rechazar los malos emparejamoentos RANSAC
    matches = matcher.match(des1, des2, None)

    points1 = np.zeros((len(matches),2), dtype=np.float32)
    points2 = np.zeros((len(matches),2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i,:] = kp1[match.queryIdx].pt
        points2[i,:] = kp2[match.trainIdx].pt

    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC) # RANSAC -> Random sample consensus
    height, width, channels = im2.shape

    im1Reg = cv2.warpPerspective(im1, h, (width, height))
    img3 = cv2.drawMatches(img1, kp1, im2, kp2, matches[:10], None, flags=4)
    
    # Si se quiere una imagen que muestre los emparejamientos entre imagenes descomentar abajo
    
    # cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
    
    cv2.imwrite(f"{name}/HG{str(nOrbs)}-{str(num)}.jpg", im1Reg)


In [ ]:
import glob
NumeroImagenes = len(glob.glob1(name+"/","image*"))
print(tifCounter)

In [ ]:


# imagen de referencia, en este caso la primera
im2 = cv2.imread(name+"/image1.jpg") # imagen de referencia
primeraImagen = cv2.cvtColor(im2,1)
img2 = cv2.cvtColor(im2,cv2.COLOR_BGR2GRAY)

# guardar la primera imagen con la nomenclatura de las otras
cv2.imwrite(f"{name}/HG{str(200)}-0.jpg", primeraImagen)

for i in range(1,NumeroImagenes+1):
    x = cv2.imread(name+"/image"+str(i)+".jpg") # imagen que se va contrastar con la de referencia
    hacerHomografia(img2,x,i, name=name)




## Hacer GIF
### Gif sin Homografía

In [ ]:
# ALINEAR LOS FOTOGRAMAS SIN HOMOGRAFIA

images = []
for i in range(1,NumeroImagenes+1):
    num = int(str(i))
    images.append(imageio.imread(name+"/image"+str(num)+".jpg"))
imageio.mimsave('primerGIF.gif', images)

In [ ]:
from IPython.display import Image
Image(filename="primerGIF.gif")

### Gif con Homografía

In [ ]:

images = []
for i in range(0,NumeroImagenes):

    num = int(str(i))
    images.append(imageio.imread(name+"/HG200-"+str(num)+".jpg"))
imageio.mimsave('segundoGIF.gif', images)

In [ ]:
from IPython.display import Image
Image(filename="segundoGIF.gif")

### Todo perfecto menos este fotograma, número 18

En este por alguna razón los puntos de referencia (ORB) no pudieron encontrar matchs suficientes y deformaron la imagen en lugar de alinearla

In [ ]:
from IPython.display import Image
Image(filename=name+'/HG200-18.jpg') 

### Descartarlo manualmente, sabiendo que es la imagen numero 18

In [ ]:

images = []

for i in range(0,27):
    if i==18:
        continue # esquivar el 18
    num = int(str(i))
    images.append(imageio.imread(name+"/HG200-"+str(num)+".jpg"))
imageio.mimsave('tercerGIF.gif', images)

In [ ]:
from IPython.display import Image
Image(filename="tercerGIF.gif")

## Usando transformada de Hough para extraer el anómalo

In [ ]:
# aqui se agrupan las imagenes 
images = []
# nombre de la carpeta donde estan los fotogramas y la homografía

for i in range(0,27): # por i en las imagenes con homografía
    try:
        #leer la imagen
        img = cv2.imread(name+"/HG200-"+str(i)+".jpg", cv2.IMREAD_COLOR)
        # Convertir la imagen a escala de grises
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Dejar la imagen con sus 
        edges = cv2.Canny(gray, 50, 200)
        # Detectar puntos que forman una imagen
        lines = cv2.HoughLinesP(edges, 1, np.pi/180,threshold = 500, minLineLength=100, maxLineGap=250)
        # Dibujar lineas en una imagen llamada error
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 3)

        cv2.imwrite("error"+str(i)+".jpg", img) # guarda la imagen anómala
        continue
    except:
        pass

    num = int(str(i))
    images.append(imageio.imread(name+"/HG200-"+str(num)+".jpg"))
imageio.mimsave('cuartoGIF.gif', images) #guardar como cuartoGIF.gif

## Cuarto GIF - FINAL

In [ ]:
from IPython.display import Image
Image(filename="cuartoGIF.gif")